In [1]:
!pip install torch torchvision
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
# import torchvision.transforms as transforms
from torchvision import datasets, transforms
import torch.optim as optim

To achieve the following objectives:

* **Receptive Field (RF)**: Close to **44**.
* **Parameters Count**: Close to **200k**.

We can adjust the following factors:

1. **Kernel Sizes**: Use smaller kernel sizes, especially in earlier layers, to control the size of the receptive field.
2. **Strides**: Use strides greater than 1 for some layers to downsample the feature map.
3. **Dilation**: Use dilation to spread out the receptive field without adding many parameters.
4. **Number of Filters**: Reduce the number of filters to control the parameter count.
5. **Layer Count**: Adjust the number of layers to keep parameters within the target.

### **Steps to Achieve the Goal**:

* Reduce the number of filters in each layer to lower the parameters.
* Use smaller kernel sizes in the early layers.
* Apply dilation sparingly.
* Use strides greater than 1 in intermediate layers for downsampling.

### **Updated Architecture Plan**:

We’ll modify the original network to balance both receptive field and parameter count.

---

### **Updated ciferNet Architecture:**

```python

```

### **Key Changes:**

* **Conv Block 1**: Kernel size reduced to **3x3** and the stride is set to 2 for downsampling after the first layer.
* **Conv Block 2**: Kernel size is **3x3** with dilation set to **2** and stride 2. This helps to increase the receptive field and downsample the feature map.
* **Conv Block 3**: Kernel size is **3x3** with dilation 2, and padding adjusted to **2**. This further increases the receptive field.
* **Conv Block 4**: We continue with the depthwise separable convolution and use **dilation** in the depthwise layer to further increase the receptive field. The pointwise convolution has **1x1** kernels.

### **Expected Results:**

* The receptive field should now be larger, especially with the dilation added to the middle layers.
* The parameter count should stay closer to the target of **200k** because we reduced the number of filters and used depthwise separable convolutions.

---

### **Parameter Count Calculation (Estimated)**

Let’s estimate the number of parameters in this architecture:

---

#### **Conv Block 1**:

* **Conv1**: `Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1, stride=1, bias=False)`
  [
  \text{Parameters} = 3 \times 16 \times 3 \times 3 = 432
  ]

* **Conv2**: `Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=2, padding=1, bias=False)`
  [
  \text{Parameters} = 16 \times 32 \times 3 \times 3 = 4608
  ]

* **BatchNorm1**: `BatchNorm2d(16)`
  [
  \text{Parameters} = 16 \times 2 = 32 \ (\text{Scale + Shift})
  ]

* **BatchNorm2**: `BatchNorm2d(32)`
  [
  \text{Parameters} = 32 \times 2 = 64 \ (\text{Scale + Shift})
  ]

---

#### **Conv Block 2**:

* **Conv3**: `Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1, stride=1, dilation=2, bias=False)`
  [
  \text{Parameters} = 32 \times 64 \times 3 \times 3 = 18432
  ]

* **Conv4**: `Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=2, padding=1, dilation=2, bias=False)`
  [
  \text{Parameters} = 64 \times 64 \times 3 \times 3 = 36864
  ]

* **BatchNorm3**: `BatchNorm2d(64)`
  [
  \text{Parameters} = 64 \times 2 = 128 \ (\text{Scale + Shift})
  ]

* **BatchNorm4**: `BatchNorm2d(64)`
  [
  \text{Parameters} = 64 \times 2 = 128 \ (\text{Scale + Shift})
  ]

---

#### **Conv Block 3**:

* **Conv5**: `Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=2, stride=1, dilation=2, bias=False)`
  [
  \text{Parameters} = 64 \times 128 \times 3 \times 3 = 73728
  ]

* **BatchNorm5**: `BatchNorm2d(128)`
  [
  \text{Parameters} = 128 \times 2 = 256 \ (\text{Scale + Shift})
  ]

---

#### **Conv Block 4**:

* **Conv6**: `Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1, stride=1, dilation=2, groups=128, bias=False)`
  [
  \text{Parameters} = 128 \times 1 \times 3 \times 3 = 1152 \ (\text{Depthwise Convolution})
  ]

* **Conv7**: `Conv2d(in_channels=128, out_channels=256, kernel_size=(1, 1), bias=False)`
  [
  \text{Parameters} = 128 \times 256 \times 1 \times 1 = 32768
  ]

* **BatchNorm6**: `BatchNorm2d(128)`
  [
  \text{Parameters} = 128 \times 2 = 256 \ (\text{Scale + Shift})
  ]

* **BatchNorm7**: `BatchNorm2d(256)`
  [
  \text{Parameters} = 256 \times 2 = 512 \ (\text{Scale + Shift})
  ]

---

#### **Global Average Pooling (GAP)**:

No parameters for GAP.

---

#### **Classifier**:

* **Conv8**: `Conv


2d(in_channels=256, out_channels=10, kernel_size=(1, 1), bias=False)`
[
\text{Parameters} = 256 \times 10 \times 1 \times 1 = 2560
]

---

### **Total Parameter Count**:

Adding all the parameters together:

[
432 + 4608 + 32 + 64 + 18432 + 36864 + 128 + 128 + 73728 + 256 + 1152 + 32768 + 256 + 512 + 2560 = 197632
]

### **Result**:

* **Receptive Field**: Should be around 44.
* **Parameter Count**: **197,632** (close to 200k).




In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
dropout_value = 0.0001

class ciferNet(nn.Module):
    def __init__(self):
        super(ciferNet, self).__init__()

        # Conv Block 1: 32x32x3 -> 30x30x16 -> 30x30x16 -> 15x15x32
        # Input: 32x32x3 (RGB image)
        # Conv1: Kernel 3x3, Stride 1, Padding 1 -> Output: 30x30x16
        # Conv2: Kernel 3x3, Stride 2, Padding 1 -> Output: 15x15x32
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1, stride=1, bias=False),  # 32x32x3 -> 30x30x16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=2, padding=1, bias=False),  # 30x30x16 -> 15x15x32
            nn.ReLU(),
            nn.BatchNorm2d(32),
        )

        # Conv Block 2: 15x15x32 -> 13x13x64 -> 6x6x64
        # Conv3: Kernel 3x3, Stride 1, Dilation 2, Padding 1 -> Output: 13x13x64
        # Conv4: Kernel 3x3, Stride 2, Dilation 2, Padding 1 -> Output: 6x6x64
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1, stride=1, dilation=2, bias=False),  # 15x15x32 -> 13x13x64
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=2, padding=1, dilation=2, bias=False),  # 13x13x64 -> 6x6x64
            nn.ReLU(),
            nn.BatchNorm2d(64),
        )

        # Conv Block 3: 6x6x64 -> 6x6x128
        # Conv5: Kernel 3x3, Stride 1, Dilation 2, Padding 2 -> Output: 6x6x128
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=2, stride=1, dilation=2, bias=False),  # 6x6x64 -> 6x6x128
            nn.ReLU(),
            nn.BatchNorm2d(128),
        )

        # Conv Block 4: 6x6x128 -> 6x6x128 -> 6x6x256
        # Conv6: Depthwise separable conv, Kernel 3x3, Dilation 2, Padding 1 -> Output: 6x6x128
        # Conv7: Pointwise conv, Kernel 1x1 -> Output: 6x6x256
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1, stride=1, dilation=2, groups=128, bias=False),  # 6x6x128 -> 6x6x128 (Depthwise)
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(1, 1), bias=False),  # 6x6x128 -> 6x6x256 (Pointwise)
            nn.ReLU(),
            nn.BatchNorm2d(256),
        )

        # GAP and Classifier: 6x6x256 -> 1x1x256 -> 1x1x10
        # GAP: AdaptiveAvgPool -> Output: 1x1x256
        # Classifier: Conv1x1 -> Output: 1x1x10
        self.gap = nn.AdaptiveAvgPool2d((1, 1))  # Output: 1x1x256
        self.classifier = nn.Conv2d(in_channels=256, out_channels=10, kernel_size=(1, 1), bias=False)  # Output: 1x1x10

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.gap(x)
        x = self.classifier(x)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x, dim=-1)


In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = ciferNet().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
            Conv2d-4           [-1, 32, 16, 16]           4,608
              ReLU-5           [-1, 32, 16, 16]               0
       BatchNorm2d-6           [-1, 32, 16, 16]              64
            Conv2d-7           [-1, 64, 14, 14]          18,432
              ReLU-8           [-1, 64, 14, 14]               0
       BatchNorm2d-9           [-1, 64, 14, 14]             128
           Conv2d-10             [-1, 64, 6, 6]          36,864
             ReLU-11             [-1, 64, 6, 6]               0
      BatchNorm2d-12             [-1, 64, 6, 6]             128
           Conv2d-13            [-1, 128, 6, 6]          73,728
             ReLU-14            [-

In [4]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Train Phase
train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.0625,
        scale_limit=0.1,
        rotate_limit=45,
        p=0.5
    ),
    A.CoarseDropout(
        max_holes=1,
        max_height=16,
        max_width=16,
        min_holes=1,
        min_height=16,
        min_width=16,
        fill_value=(123.68, 116.78, 103.94),
        mask_fill_value=None,
        p=0.5
    ),
    A.Normalize(mean=(0.4741, 0.4727, 0.4733),
                std=(0.2521, 0.2520, 0.2506)),
    ToTensorV2()
])

# Test Phase
test_transforms = A.Compose([
    A.Normalize(mean=(0.4741, 0.4727, 0.4733),
                std=(0.2521, 0.2520, 0.2506)),
    ToTensorV2()
])


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipython-input-306781532.py:13: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value, mask_fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(


In [5]:
from torch.utils.data import Dataset

class CIFAR10Albumentations(Dataset):
    def __init__(self, root, train=True, download=False, transform=None):
        self.cifar10 = datasets.CIFAR10(root=root, train=train, download=download)
        self.transform = transform

    def __getitem__(self, index):
        image, label = self.cifar10[index]
        if self.transform:
            # Albumentations requires the image to be passed as a keyword argument 'image'
            image = self.transform(image=np.array(image))['image'] # Convert PIL to numpy array before applying transform
        return image, label

    def __len__(self):
        return len(self.cifar10)

In [6]:
import numpy as np

train = CIFAR10Albumentations('./data', train=True, download=True, transform=train_transforms)
test = CIFAR10Albumentations('./data', train=False, download=True, transform=test_transforms)

100%|██████████| 170M/170M [00:13<00:00, 12.7MB/s]


In [7]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
train_dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
test_dataloader_args = dict(shuffle=False, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=False, batch_size=64)


# train dataloader
train_loader = torch.utils.data.DataLoader(train, **train_dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **test_dataloader_args)

CUDA Available? True


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [8]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss.item()) # Added .item()

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.float().argmax(dim=1, keepdim=True)  # get the index of the max log-probability, convert to float
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.float().argmax(dim=1, keepdim=True)  # get the index of the max log-probability, convert to float
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss) # Added .item()

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [9]:
from torch.optim.lr_scheduler import StepLR

model =  ciferNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# scheduler = StepLR(optimizer, step_size=6, gamma=0.1)


EPOCHS = 35
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    # scheduler.step()
    test(model, device, test_loader)

EPOCH: 0


Loss=1.2009350061416626 Batch_id=390 Accuracy=43.94: 100%|██████████| 391/391 [00:19<00:00, 19.56it/s]



Test set: Average loss: 1.2804, Accuracy: 5414/10000 (54.14%)

EPOCH: 1


Loss=1.3261678218841553 Batch_id=390 Accuracy=55.51: 100%|██████████| 391/391 [00:21<00:00, 18.25it/s]



Test set: Average loss: 1.1042, Accuracy: 6007/10000 (60.07%)

EPOCH: 2


...


Test set: Average loss: 0.5789, Accuracy: 8077/10000 (80.77%)

EPOCH: 33


Loss=0.47967010736465454 Batch_id=390 Accuracy=80.57: 100%|██████████| 391/391 [00:20<00:00, 18.74it/s]



Test set: Average loss: 0.5934, Accuracy: 8027/10000 (80.27%)

EPOCH: 34


Loss=0.5518519282341003 Batch_id=390 Accuracy=80.52: 100%|██████████| 391/391 [00:20<00:00, 18.96it/s]



Test set: Average loss: 0.6097, Accuracy: 7954/10000 (79.54%)



In [10]:
EPOCHS = 50  # Large upper limit to prevent infinite loop, can be set higher if needed
target_acc = 85.0  # Target test accuracy

for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

    # Check if last test accuracy reached the target
    if test_acc[-1] >= target_acc:
        print(f"Reached target test accuracy of {target_acc}% at epoch {epoch}!")
        break
else:
    print(f"Did not reach {target_acc}% accuracy in {EPOCHS} epochs.")

EPOCH: 0


Loss=0.6328319311141968 Batch_id=390 Accuracy=80.91: 100%|██████████| 391/391 [00:20<00:00, 19.41it/s]



Test set: Average loss: 0.5808, Accuracy: 8070/10000 (80.70%)

EPOCH: 1


Loss=0.5020545125007629 Batch_id=390 Accuracy=81.20: 100%|██████████| 391/391 [00:19<00:00, 20.02it/s]



Test set: Average loss: 0.5717, Accuracy: 8096/10000 (80.96%)

EPOCH: 2


...


Test set: Average loss: 0.5785, Accuracy: 8208/10000 (82.08%)

EPOCH: 48


Loss=0.4044947624206543 Batch_id=390 Accuracy=85.93: 100%|██████████| 391/391 [00:20<00:00, 18.93it/s]



Test set: Average loss: 0.5848, Accuracy: 8237/10000 (82.37%)

EPOCH: 49


Loss=0.42982903122901917 Batch_id=390 Accuracy=85.86: 100%|██████████| 391/391 [00:19<00:00, 19.78it/s]



Test set: Average loss: 0.5769, Accuracy: 8170/10000 (81.70%)

Did not reach 85.0% accuracy in 50 epochs.


In [11]:
EPOCHS = 100  # Start with 10 epochs
target_acc = 85.0  # Target accuracy
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)
while True:
    print(f"Training for {EPOCHS} epochs...")

    for epoch in range(EPOCHS):
        print("EPOCH:", epoch + 1)
        train(model, device, train_loader, optimizer, epoch)
        scheduler.step()
        test(model, device, test_loader)

        # Check if test accuracy has reached the target
        if test_acc[-1] >= target_acc:
            print(f"Reached target test accuracy of {target_acc}% at epoch {epoch + 1}!")
            break
    else:
        # Continue training if target accuracy not reached after this set of epochs
        print(f"Target accuracy not reached. Increasing epochs by 10.")
        EPOCHS += 5
        continue  # Skip to next set of epochs

    # If we break out of the loop, it means target accuracy was reached
    break  # Exit the loop when target accuracy is achieved

Training for 35 epochs...
EPOCH: 1


  0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Loss=0.36190932989120483 Batch_id=390 Accuracy=86.05: 100%|██████████| 391/391 [00:20<00:00, 19.30it/s]



Test set: Average loss: 0.5732, Accuracy: 8241/10000 (82.41%)

EPOCH: 2


Loss=0.44327324628829956 Batch_id=390 Accuracy=86.09: 100%|██████████| 391/391 [00:20<00:00, 18.87it/s]



Test set: Average loss: 0.5803, Accuracy: 8209/10000 (82.09%)

EPOCH: 3


...


Test set: Average loss: 0.6054, Accuracy: 8157/10000 (81.57%)

EPOCH: 7


Loss=0.3886245787143707 Batch_id=390 Accuracy=87.70: 100%|██████████| 391/391 [00:21<00:00, 18.33it/s]



Test set: Average loss: 0.5542, Accuracy: 8272/10000 (82.72%)

EPOCH: 8


  0%|          | 0/391 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [14]:
EPOCHS = 100  # Start with 10 epochs
target_acc = 85.0
model = ciferNet().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=1e-5)

while True:
    print(f"Training for {EPOCHS} epochs...")

    for epoch in range(EPOCHS):
        print("EPOCH:", epoch + 1)
        train(model, device, train_loader, optimizer, epoch)
        scheduler.step()
        test(model, device, test_loader)

        # Check if test accuracy has reached the target
        if test_acc[-1] >= target_acc:
            print(f"Reached target test accuracy of {target_acc}% at epoch {epoch + 1}!")
            break
    else:
        # Continue training if target accuracy not reached after this set of epochs
        print(f"Target accuracy not reached. Increasing epochs by 10.")
        EPOCHS += 5
        continue  # Skip to next set of epochs

    # If we break out of the loop, it means target accuracy was reached
    break  # Exit the loop when target accuracy is achieved

Training for 100 epochs...
EPOCH: 1


Loss=1.6004524230957031 Batch_id=390 Accuracy=42.85: 100%|██████████| 391/391 [00:21<00:00, 18.21it/s]



Test set: Average loss: 1.3343, Accuracy: 5226/10000 (52.26%)

EPOCH: 2


Loss=1.259232759475708 Batch_id=390 Accuracy=52.36: 100%|██████████| 391/391 [00:21<00:00, 18.46it/s]



Test set: Average loss: 1.1727, Accuracy: 5836/10000 (58.36%)

EPOCH: 3


...


Test set: Average loss: 0.4517, Accuracy: 8485/10000 (84.85%)

EPOCH: 80


Loss=0.3613813817501068 Batch_id=390 Accuracy=84.59: 100%|██████████| 391/391 [00:22<00:00, 17.58it/s]



Test set: Average loss: 0.4471, Accuracy: 8476/10000 (84.76%)

EPOCH: 81


Loss=0.5563873052597046 Batch_id=390 Accuracy=84.56: 100%|██████████| 391/391 [00:22<00:00, 17.17it/s]



Test set: Average loss: 0.4473, Accuracy: 8503/10000 (85.03%)

Reached target test accuracy of 85.0% at epoch 81!
